In [1]:
import csv
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import math

In [3]:
#general model
provinceChoice = 'western'
days_in_future = 1
time_step = []
cases = []
model = load_model(f'../models/province/lstm-model-{provinceChoice}-prov.h5')

provinces = {'central': [5, 15], 'eastern': [5, 7], 'northcentral': [5, 40], 'northwestern': [5, 33], 'northern': [5, 10],
             'sabaragamuwa': [5, 13], 'southern': [5, 13], 'uva': [5, 9], 'western': [20, 110]
            }

window_size = provinces[provinceChoice][0]
split_time = provinces[provinceChoice][1]


def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

with open(f'../final province csvs/{provinceChoice}_province.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  steps = 0
  for row in reader:
    cases.append(float(row[-1]))
    time_step.append(steps)
    steps+=1

for i in range(days_in_future):
    time_step.append(time_step[-1] + 1)

for i in range(days_in_future):
    cases.append(0)

series = np.array(cases)
time = np.array(time_step)

rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

prediction = math.ceil(rnn_forecast[-1])

if prediction == 1:
    print(math.ceil(rnn_forecast[-1]), "case is predicted for " + str(days_in_future) + f" days in future in the {provinceChoice} Province")
else:
    print(math.ceil(rnn_forecast[-1]), "cases are predicted for " + str(days_in_future) + f" days in future in the {provinceChoice} Province")

4 cases are predicted for 1 days in future in the western Province
